# Installing Libraries and Imports

In [1]:
!pip install -U accelerate
!pip install -U transformers

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 309.4/309.4 kB 6.4 MB/s eta 0:00:00
  Using cached nvidia_cuda_nvrtc_cu12-12.1.105-py3-none-manylinux1_x86_64.whl (23.7 MB)
  Using cached nvidia_cuda_runtime_cu12-12.1.105-py3-none-manylinux1_x86_64.whl (823 kB)
  Using cached nvidia_cuda_cupti_cu12-12.1.105-py3-none-manylinux1_x86_64.whl (14.1 MB)
  Using cached nvidia_cudnn_cu12-8.9.2.26-py3-none-manylinux1_x86_64.whl (731.7 MB)
  Using cached nvidia_cublas_cu12-12.1.3.1-py3-none-manylinux1_x86_64.whl (410.6 MB)
  Using cached nvidia_cufft_cu12-11.0.2.54-py3-none-manylinux1_x86_64.whl (121.6 MB)
  Using cached nvidia_curand_cu12-10.3.2.106-py3-none-manylinux1_x86_64.whl (56.5 MB)
  Using cached nvidia_cusolver_cu12-11.4.5.107-py3-none-manylinux1_x86_64.whl (124.2 MB)
  Using cached nvidia_cusparse_cu12-12.1.0.106-py3-none-manylinux1_x86_64.whl (196.0 MB)
  Using cached nvidia_nccl_cu12-2.20.5-py3-none-manylinux2014_x86_64.whl (176.2 MB)
  Using cached nvidia_nvtx_cu12-12.1.105-py3-none-m

In [2]:
import torch
from transformers import AutoModelForCausalLM, AutoTokenizer, pipeline

In this notebook, you will be working with a Large Language Model (LLM) and explore its capabilities to help you solve various problems.

# Loading Model

We will be using Phi-3 as our LLM.

In [3]:
MODEL_ARGS = {
    'Name': 'microsoft/Phi-3-mini-128k-instruct',
    'DType': torch.bfloat16 # add torch.
}
device = "cuda" if torch.cuda.is_available() else "cpu"

In [4]:
def load_model(model_args):


    model = AutoModelForCausalLM.from_pretrained(
        model_args['Name'],
        trust_remote_code=True,
        torch_dtype=model_args['DType'], #remove torch.
        low_cpu_mem_usage=True,
        device_map={"": device},
    )
    tokenizer = AutoTokenizer.from_pretrained(
        model_args['Name'],
        trust_remote_code=True,
    )

    return model, tokenizer

In [5]:
model, tokenizer = load_model(MODEL_ARGS)

/usr/local/lib/python3.10/dist-packages/huggingface_hub/utils/_token.py:89: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(


config.json:   0%|          | 0.00/3.38k [00:00<?, ?B/s]

configuration_phi3.py:   0%|          | 0.00/10.4k [00:00<?, ?B/s]

A new version of the following files was downloaded from https://huggingface.co/microsoft/Phi-3-mini-128k-instruct:
- configuration_phi3.py
. Make sure to double-check they do not contain any added malicious code. To avoid downloading new versions of the code file, you can pin a revision.


modeling_phi3.py:   0%|          | 0.00/73.8k [00:00<?, ?B/s]

...

tokenizer.model:   0%|          | 0.00/500k [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/1.84M [00:00<?, ?B/s]

added_tokens.json:   0%|          | 0.00/293 [00:00<?, ?B/s]

special_tokens_map.json:   0%|          | 0.00/568 [00:00<?, ?B/s]

Special tokens have been added in the vocabulary, make sure the associated word embeddings are fine-tuned or trained.


# First Inference

In [6]:
def generate_text(model, tokenizer, prompt, max_new_tokens = 100, do_sample=True, temperature=0.5):

    input_ids = tokenizer.encode(prompt, return_tensors='pt').to(device)
    if do_sample:
        output_ids = model.generate(input_ids, max_new_tokens=max_new_tokens, do_sample=True, temperature=temperature)
    else:
        output_ids = model.generate(input_ids, max_new_tokens=max_new_tokens, do_sample=do_sample)

    output_text = tokenizer.decode(output_ids[0], skip_special_tokens=True)

    return output_text[len(prompt):]

Lets break down this function:

**Arguments**:

* **model**: The language model used for text generation.
* **tokenizer**: The tokenizer that converts text to tokens and vice versa.
* **prompt**: The initial text input that the model will build upon.
* **max_new_tokens**: The maximum number of new tokens to generate.
* **do_sample**: Whether to sample the next token or use deterministic decoding.
* **temperature**: Controls the randomness of sampling; higher values produce more diverse outputs. ( model creativity )

**Functionality**:

The generate_text function creates more text based on a given starting prompt using a language model and tokenizer. It first converts the prompt into tokens (numbers the model understands), then generates additional tokens to continue the text. Depending on settings, it can generate text randomly or in a fixed way. Finally, it converts the tokens back into readable text and returns the part that extends beyond the original prompt.

## Without template

In [7]:
prompt = """Tell me a funny story about a cute cat"""

generate_text(
    model=model,
    tokenizer=tokenizer,
    prompt=prompt,
    max_new_tokens=200,
    temperature=1.2,
)

''

## With template

In [12]:
prompt = """Insturction: Tell me a funny story about a cute cat
Answer:"""

generate_text(
    model=model,
    tokenizer=tokenizer,
    prompt=prompt,
    max_new_tokens=200,
    temperature=1.2,
)

" Once upon a time, there was a cute and playful cat named Miss Fluffernisspuff. She lived with an old lady who always told people that Miss Fluffernisspuff was exceptionally intelligent. One day, the old lady decided to test if her claim was true.\n\nIn the midst of their afternoon chillaxation, the old lady left for a brief moment to fetch some tea. Seizing the opportunity, Miss Fluffernisspuff grabbed an old, wrinkled shirt from the laundry basket and, with unmatched elegance and skill, started to do her own version of the ‘lady's laundry dance routine,' clawing, stitching, ruffling, and wrinkling the shirt simultaneously. Of course, it looked a disaster.\n\nWhen the old lady returned home, she saw her beloved cat wearing the shirt like a cape"

As you can see, the output generated by these models depends on the prompt provided.

But that's just the beginning! Let's try different prompt layouts

( You can use the keyword "Prompt Engineering" for more information )

# In Context Learning ( ICL )

LLMs can learn from their prompts, as you can give it examples or guide it and teach it how to solve the problem.

## Learning from examples

### No example

In [9]:

prompt = """Question: John volunteers at a shelter once a week for 3 hours at a time. How many hours does he volunteer per year?
Answer:"""

generate_text(
    model=model,
    tokenizer=tokenizer,
    prompt=prompt,
    max_new_tokens=200,
    temperature=0.2,
)

' John volunteers 192 hours per year.'

The right answer is ( 12 * 4 ) * 3 = 144

### One Example

In [10]:

prompt = """Question: John volunteers at a shelter once a week for 7 hours at a time. How many hours does he volunteer per year?
Answer: John volunteers 336 hours per year.
Question: John volunteers at a shelter once a week for 3 hours at a time. How many hours does he volunteer per year?
Answer:"""

generate_text(
    model=model,
    tokenizer=tokenizer,
    prompt=prompt,
    max_new_tokens=10,
    do_sample=False,
    temperature=0.0,
)

' John volunteers 156 hours per year'

Examples are not always effective for mathematical problems, so let's try another method.

## Chain of Thoughts ( CoT )

In [11]:

prompt = """Question: John volunteers at a shelter once a week for 7 hours at a time. How many hours does he volunteer per year?
Answer: There are 12 months in one year and 4 weeks in each month. So in one year, there are 12 * 4 = 48 weeks. If Jhon volunteers at a shelter once a week for 7 hours,
John volunteers 48 * 7 = 336 hours per year.
Question: John volunteers at a shelter once a week for 3 hours at a time. How many hours does he volunteer per year?
Answer:"""

generate_text(
    model=model,
    tokenizer=tokenizer,
    prompt=prompt,
    max_new_tokens=70,
    do_sample=False,
    temperature=0.0,
)

' There are 12 months in one year and 4 weeks in each month. So in one year, there are 12 * 4 = 48 weeks. If John volunteers at a shelter once a week for 3 hours,\nJohn volunteers 48 * 3 = 144 hours per year.'

In Chain of Thought (CoT), we guide the model with one or more examples and provide it with the steps to solve the problem.

In [14]:
import pandas as pd

In [15]:
df_train = pd.read_csv("/content/CSVs/train.csv")
df_test = pd.read_csv("/content/CSVs/test.csv")
df_unlabeled = pd.read_csv("/content/CSVs/unlabeled.csv")

In [38]:
def generate_label(review_text, model, tokenizer, max_length=512, do_sample=True,temperature=0.5):
    prompt = f"""
    Question: i will give you a review and if it is positive you answer with Positive and if Negative you answer with Negative: This has to be one of my favourite movies of all time. The dialogue, with the constant use of puns is very tight, the cast are superb, and the plot is highly original. Don't take my word for it - watch this movie and enjoy it for yourself.
    Answer: Positive
    Question: i will give you a review and if it is positive you answer with Positive and if Negative you answer with Negative: 'needed an excuse to get out of the house while paint dried - left the movie after an hour to return and watch the paint dry. I don't recall ever walking out on a movie before, but I really tried to stay. The script was not up to the cast and just kept 'going and going' badly - come on! Uma Thurman doing this stuff? Fairly lame special effects. These were older characters and actors doing superficial horny 20-something lives - just sort of annoying and wrong feeling. This review is based only on the first hour - it might have gotten better. I just had to get home and see if the paint dried a darker shade than when it went on.'
    Answer: Negative
    Question: i will give you a review and if it is positive you answer with Positive and if Negative you answer with Negative: 'this film was the worst film i have ever viewed. it was like a 'homework assignment' for a film class. it totally misses the mark when it comes to the 'message' it is trying to relay. characters are over exaggerated, poor acting and as for a plot...well it is utterly ridiculous. the cover shot is what made me think it may be a decent film, the co-actor is handsome and that's about it. moral of this movie: never judge a movie by its cover! save your time, money and energy and make your own home movie and you will be far better off than i. it was painful to watch and quite frankly i am surprised that anyone would spend money to make and distribute it!'
    Answer: Negative
    Question: i will give you a review and if it is positive you answer with Positive and if Negative you answer with Negative: {review_text}
    Answer:"""

    generated_text_indiced = generate_text(
        model=model,
        tokenizer=tokenizer,
        prompt=prompt,
        max_new_tokens=50,
        do_sample=False,
        temperature=0.0,
    )
    # print(generated_text_indiced)

    if "Positive" in generated_text_indiced:
        return 1
    elif "Negative" in generated_text_indiced:
        return 0
    else:
        return None  # Unable to classify

In [40]:
df_unlabeled["text"][40]

"in fact,it's basically the same movie.and they couldn't even get the time line of events correct.maybe that was intentional due to laziness or not caring.either way,this thing is a real woofer.it doesn't even deserved to be called a movie.i viewed this as a so called second feature on the disc containing the original the boogeyman.i thought my head would explode,and i urge you to run as far in the opposite direction of this thing,if you should be cursed with the misfortune of combing across it's path.it should come with a warning label like:warning-may cause your i.q to drop several points if you are within it's vicinity.for me,there's no doubt this thing is a 0/10"

In [44]:
# generate_label(df_unlabeled["text"][0], model, tokenizer)
# df_unlabeled
df_unlabeled['label'] = df_unlabeled['text'].apply(lambda x: generate_label(x, model, tokenizer))
df_unlabeled = df_unlabeled.dropna(subset=['label'])
# label = generate_label(df_unlabeled["text"][40], model, tokenizer, temperature=1.2)
# print(df_unlabeled["text"][0])
# print(f"Generated label: {label}")

 Positive
    Question: i will give you a review and if it is positive you answer with Positive and if Negative you answer with Negative: 'i have seen this movie 3 times now and i still enjoy it. the acting
 Negative
    Question: i will give you a review and if it is positive you answer with Positive and if Negative you answer with Negative: 'i have seen this movie many times and i have never been disappointed. the acting
 Positive
    Question: i will give you a review and if it is positive you answer with Positive and if Negative you answer with Negative: 'i have seen this movie 3 times and i still enjoy it. the story is
 Positive
    Question: i will give you a review and if it is positive you answer with Positive and if Negative you answer with Negative: 'i have seen this movie 3 times now and i still enjoy it. the acting
 Negative
    Question: i will give you a review and if it is positive you answer with Positive and if Negative you answer with Negative: 'i have seen this movie

KeyboardInterrupt: 

In [43]:
from sklearn.linear_model import LogisticRegression
from sklearn.tree import DecisionTreeClassifier
from sklearn.ensemble import RandomForestClassifier
from sklearn.svm import SVC

def train_and_evaluate_classifiers(X_train, y_train, X_test, y_test):
    classifiers = {
        "Logistic Regression": LogisticRegression(max_iter=1000, random_state=42),
        "Decision Tree": DecisionTreeClassifier(random_state=42),
        "Random Forest": RandomForestClassifier(random_state=42),
        "Support Vector Machine": SVC(probability=True, random_state=42)
    }

    scores_list = []
    for name, clf in classifiers.items():
        clf.fit(X_train, y_train)
        y_pred = clf.predict(X_test)
        accuracy = accuracy_score(y_test, y_pred)
        precision = precision_score(y_test, y_pred)
        recall = recall_score(y_test, y_pred)
        f1 = f1_score(y_test, y_pred)
        roc_auc = roc_auc_score(y_test, clf.predict_proba(X_test)[:, 1])
        scores_list.append([name, accuracy, precision, recall, f1, roc_auc])

    headers = ["Classifier", "Accuracy", "Precision", "Recall", "F1 Score", "AUC-ROC"]
    colored_headers = [Fore.GREEN + header + Fore.RESET for header in headers]
    def format_score(score):
        if isinstance(score, float):
            return Fore.YELLOW + "{:.6f}".format(score) + Fore.RESET
        else:
            return Fore.YELLOW + str(score) + Fore.RESET

    colored_scores_list = [[format_score(score) for score in row] for row in scores_list]
    print(Fore.CYAN + Style.BRIGHT + "Classifier Evaluation Scores" + Style.RESET_ALL)
    print(tabulate(colored_scores_list, headers=colored_headers, tablefmt="fancy_grid", numalign="center", stralign="center", missingval="-"))

In [ ]:
df_combined = pd.concat([df_train, df_unlabeled], ignore_index=True)
train_and_evaluate_classifiers(df_combined["text"], df_combined["label"], df_test["text"], df_test["label"])
